## Chapter 15: Calculus and automatic differentiation 

# 15.6  The reverse mode of automatic differentiation

While the forward mode of automatic differentiation introduced in the previous Section provides a programmable way of computing derivatives, it can be ineffecient for many kinds multi-input functions (particularly those in machine learning involving *multi-layer perceptrons*).  This is because while many nodes in the computation graph of a mult-input function may only take in a few inputs, we compute the *complete gradient* with respect to *all the inputs* at each and every node.  This leads to considerable waste since we know that the partial derivative of any node with respect to an original input of the function which it does *not* take in will always be equal to zero.  This obvious waste is the motivation for the *reverse mode of automatic differentiation*.  With the reverse mode we *only* compute the form of the partial derivatives needed at each node of the computation graph, ignoring partial derivatives tht will always be zero.  Because we very often deal with multi-input cost functions in machine learning the *reverse mode* - typically referred to as *the backpropogation algorithm* in machine learning - is often preferred over the forward mode detailed in the prior Section, and is in particular the brand of automatic differentiatior implemented in `autograd` (the Python-based automatic differentiator we recommend using - see Section  3.4). 

In [ ]:
# This code cell will not be shown in the HTML version of this notebook
#imports from custom library
import sys
sys.path.append('../../')
import autograd.numpy as np
import matplotlib.pyplot as plt
from mlrefined_libraries import calculus_library as calclib
from mlrefined_libraries import basics_library as baslib

%matplotlib notebook
from matplotlib import rcParams
rcParams['figure.autolayout'] = True

We ended the previous Section by detailing how to compute the form of the gradient at each node in the computation graph of the quadratic $g\left(\mathbf{w}\right) = w_1^2+w_2^2$.  Notice, however, that in calcualting the full gradient at each node we performed several wasteful computations - whenever partial derivatives were taken with respect to input not taken in by a node we know by default this partial derivative will always equal zero.  For example, the partial derivative $\frac{\partial}{\partial w_2}a = 0$ since $a$ is not a function of $w_2$.  Below we re-draw the computation graph of this quadratic with the gradient expressed in terms of partials at each node and with all zero partials marked. Examining this graph we can see that a good deal of the partials trivially equal zero.  Trivial zeros such as this represent waste when we initially compute them, store them, and propogate them forward through the graph when evaluating the gradient.

<figure>
<p><img src="../../mlrefined_images/calculus_images/forward_2.png" width="60%" height="auto"></p>
<figcaption> <strong>Figure 1:</strong> <em> The computation graph of the quadratic function described in the text, with zero partials marked.  
</em>
</figcaption>
</figure>

This issue becomes much more accentuated when considering functions with larger numbers of input variables. For example, in Figure 2 below we show the computational graph of the analgous quadratic function taking four inputs $g\left(\mathbf{w}\right) = w_1^2+w_2^2+w_3^2+w_4^2$. In this case more than *half* of all the gradient entries at the nodes are zero due to the fact that certain nodes are not functions of certain inputs, and hence their partial derivatives are zero.

<figure>
<p><img src="../../mlrefined_images/calculus_images/forward_4.png" width="100%" height="auto"></p>
<figcaption> <strong>Figure 2:</strong> <em> (left panel) Full computation graph of a simple four input quadratic with the gradient expressed at each node in terms of partial derivatives.  (right panel) The same computation graph but with all trivial partial derivatives, that is those taken with respect to input that a node is not a function of, shown as their true zero values.  Here over half of the partial derivatives computed are trivial zeros.
</em>
</figcaption>
</figure>

To remedy this inefficiency, automatic differentiation can also be performed in what is typically referred to as the *reverse mode*, which consists of a forward and reverse (or backward) sweep through the computation graph of a function.  

In the forward sweep of the reverse mode we traverse the computation graph in forward from left to right recursively just as with the forward mode, only at each node we compute the partial derivatives of each parent node with respect to *its children only* and *not* the full gradient with respect to the function input.  

This is illustrated for the quadratic function $g\left(\mathbf{w}\right) = w_1^2+w_2^2$ in the top panels below.  In the top left panel we show the computing of partial derivatives for parent nodes $a$ and $b$ (colored red), which are taken only with respect to their children (which here are $w_1$ and $w_2$ respectively, colored blue).  In the top right panel we illustrate the next computation in the forward sweep, the partial derivatives computed at the parent node $c$ (colored red) with respect to its children $a$ and $b$ (colored blue).

<figure>
<p><img src="../../mlrefined_images/calculus_images/reverse_mode_1.png" width="100%" height="auto"></p>
<p><img src="../../mlrefined_images/calculus_images/reverse_mode_2.png" width="100%" height="auto"></p>

<figcaption> <strong>Figure 2:</strong> <em> (left panel) Full computation graph of a simple four input quadratic with the gradient expressed at each node in terms of partial derivatives.  (right panel) The same computation graph but with all trivial partial derivatives, that is those taken with respect to input that a node is not a function of, shown as their true zero values.  Here over half of the partial derivatives computed are trivial zeros.
</em>
</figcaption>
</figure>

Once the forward sweep is complete, we change course and traverse *backwards* recursively through the computation graph starting at the end node.  At every step of the process we update the partial derivative of each child by multiplying it by the partial derivative of its parent node with respect to that child.  When the backward sweep is completed we will have recursively constructed the gradient of the function with respect to all of its inputs. 

The backward sweep is illustrated for the two-input quadratic function in the bottom panels of the figure above.  Starting from end node in our forward sweep, i.e., node $c$, we observe that $c$ has two children: $a$ and $b$. Therefore we update the derivative at $a$ by (left) multiplying it by $\frac{\partial}{\partial a}c$ giving $\frac{\partial}{\partial a}c \frac{\partial}{\partial w_1}a$, and similarly update the derivative at $b$ by (left) multiplying it by $\frac{\partial}{\partial b}c$ giving $\frac{\partial}{\partial b}c \frac{\partial}{\partial w_2}b$.  We then repeat this procedure recursively with the children of $a$ and $b$, ending with the partial derivative $\frac{\partial}{\partial a}c \frac{\partial}{\partial w_1}a \frac{\partial}{\partial w_1}w_1 = \frac{\partial}{\partial a}c \frac{\partial}{\partial w_1}a$ and $\frac{\partial}{\partial b}c \frac{\partial}{\partial w_2}b \frac{\partial}{\partial w_2}w_2 = \frac{\partial}{\partial b}c \frac{\partial}{\partial w_2}b $.  These are precisely the two partial derivatives of the complete gradient of the quadratic with respect to its input $w_1$ and $w_2$.